In [8]:
import os
import util
import pandas as pd
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
projects = util.select_projects_from_libraries_io()
libraries = util.select_libraries_from_libraries_io()
lib_names = set(libraries["name"])
dep_changes = None
with multiprocessing.Pool(32) as pool:
    results = pool.starmap(
        util.select_dependency_changes,
        [(proj_name, lib_names) for proj_name in projects["nameWithOwner"]]
    )
    pd.concat(filter(lambda x: x is not None, results))
len(dep_changes)

TypeError: object of type 'NoneType' has no len()